In [4]:
import requests
from bs4 import BeautifulSoup
import re
import math
import itertools
import json
import urllib
from pandas.io.json import json_normalize
import pandas as pd
import time


# Main URL - We have to get accessvariable hash code to proof we are human
# Page URL is not important, we use random page to get hash.
main_url = 'https://resistancemap.cddep.org/AntibioticResistance.php'

# Data URL - Send parameteres with hash code
data_url = 'https://resistancemap.cddep.org/getData.php'

# Years
years = [2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015]

# Add all IDs to get whole data
parameters = {
    # All Antibiotics
    'abxId[]': [
        -99, 4, 33, 5, 34, 35, 7, 41, 18, 45, 10, 42, 36, 37, 43, 44, 19, 39, 40
    ], 
    # All Countries
    'countryId[]': [
        5, 38, 17, 44, 70, 1, 76, 77, 78, 92, 82, 87, 26, 91, 93, 36, 39, 15,
        13, 14, 12, 11, 10, 9, 8, 6, 4, 98, 3, 2, 31, 89, 42, 30, 29, 28, 27,
        25, 23, 22, 21, 99, 19, 18, 72, 71, 69, 68, 67, 65, 37, 103, 43, 35,
        34, 32, 90, 84, 81, 80, 74, 79, 64, 97, 96, 88, 86, 83, 73, 75, 100,
        85, 95, 66, 94, 33
    ],  
    'charttype':'chart',
    'page':'use',
    'units':'DDD',
}


def get_source(url):
    source = requests.get(url)
    soup = BeautifulSoup(source.content, 'html.parser')
    return soup


def get_access_variable(url):
    soup = BeautifulSoup(requests.get(url).content, 'html.parser')
    scripts = soup.findAll('script')
    input = soup.findAll('script', type="text/javascript")[2]
    accessvariable = re.search(r'accessvariable: "(.*)"', str(input))[1]
    return str(accessvariable)


def save_xlsx(data, excel_name):
    writer = pd.ExcelWriter(excel_name + '.xlsx', engine='xlsxwriter')
    df.to_excel(writer,
                sheet_name='Sheet1',
                index=False,
                startrow=1,
                header=False)
    worksheet = writer.sheets['Sheet1']
    workbook = writer.book
    header_format = workbook.add_format({
        'bold': True,
        'valign': 'top',
        #'fg_color': '#ffcccc',
        'border': 0
    })
    worksheet.set_row(0, None, header_format)
    for col_num, value in enumerate(data.columns.values):
        worksheet.write(0, col_num, value, header_format)
    writer.save()


# Hash Code
parameters["accessvariable"] = get_access_variable(main_url)
parameters["year"] = years[0]

# Request and get Json
r = requests.post(data_url, data=parameters).json()
#json.dumps(r)

# Get All Json files by year
dataframes = []
for y in years:
    parameters["year"] = y
    r = requests.post(data_url, data=parameters).json()
    dataframes.append( pd.DataFrame(r[2]) )
    time.sleep(2)

# Concat all years
all = pd.concat(dataframes,ignore_index=True)

# write to excel
all.to_excel("AntibioticUse.xlsx", index=False, header=True)